In [5]:
from dotenv import get_key
from pymongo import MongoClient
import pymongo
import psycopg2 # Remover
from pprint import pprint  # Imprimir bsons formatados
from utils import *

STRING = get_key('.env', 'MG_STRING')
NAME = get_key('.env', 'PG_NAME')
USER = get_key('.env', 'PG_USER')
PASSWORD = get_key('.env', 'PG_PASSWORD')
HOST = get_key('.env', 'PG_HOST')
PORT = get_key('.env', 'PG_PORT')

## Funções para acesso ao MongoDB

In [6]:
def create_mongo_connection():
	"""Cria uma conexão com o banco de dados MongoDB. Retorna o cliente.
	AVISO: Se a conexão não funcionar, verifique no site do atlas se seu IP atual está autorizado a se conectar."""
	try:
		client = MongoClient(STRING, server_api=pymongo.server_api.ServerApi(version="1"))
		client.admin.command("ping")
		print("---- Conexão bem sucedida ----")
	except Exception as e:
		raise Exception(
			"The following error occurred: ", e)
	return client

def end_mongo_connection(client):
	client.close()
	print("---- Conexão encerrada -----")

## Criação de tabelas no banco

In [ ]:
def create_pais_collection(db, v_action, v_level):
    schema = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["sigla", "nome", "continente", "grupos", "estatisticas"],
            "properties": {
                "sigla": {"bsonType": "string"},
                "nome": {"bsonType": "string"},
                "continente": {"bsonType": "string"},
                "grupos": {
                    "bsonType": "array",
                    "items": {"bsonType": "string"}
                },
                "estatisticas": {
                    "bsonType": "array",
                    "items": {
                        "bsonType": "object",
                        "required": ["ano", "pib", "geografia"],
                        "properties": {
                            "ano": {"bsonType": "int"},
                            "pib": {"bsonType": "double"},
                            "geografia": {
                                "bsonType": "array",
                                "items": {
                                    "bsonType": "object",
                                    "required": ["area_tipo", "num_habitantes", "area_km2"],
                                    "properties": {
                                        "area_tipo": {"bsonType": "string"},
                                        "num_habitantes": {"bsonType": "int"},
                                        "area_km2": {"bsonType": "double"}
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }

    try:
        db.create_collection("pais", validator=schema, validationAction=v_action, validationLevel=v_level)
    except Exception as e:
        print(f"Erro ao criar a coleção: {e}")


In [ ]:
def create_emissao_gas_collection(db, v_action, v_level):
    schema = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["nome", "formula", "medicoes"],
            "properties": {
                "nome": {"bsonType": "string"},
                "formula": {"bsonType": "string"},
                "medicoes": {
                    "bsonType": "array",
                    "items": {
                        "bsonType": "object",
                        "required": ["cod_pais", "ano", "valor_mtco2"],
                        "properties": {
                            "cod_pais": {"bsonType": "string"},
                            "ano": {"bsonType": "int"},
                            "valor_mtco2": {"bsonType": "double"}
                        }
                    }
                }
            }
        }
    }

    try:
        db.create_collection("emissaoGas", validator=schema, validationAction=v_action, validationLevel=v_level)
    except Exception as e:
        print(f"Erro ao criar a coleção: {e}")


In [ ]:
def create_energia_collection(db, v_action, v_level):
    schema = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["tipo_interacao", "unidade", "fontes"],
            "properties": {
                "tipo_interacao": {"bsonType": "string"},
                "unidade": {"bsonType": "string"},
                "fontes": {
                    "bsonType": "array",
                    "items": {
                        "bsonType": "object",
                        "required": ["nome", "medicoes"],
                        "properties": {
                            "nome": {"bsonType": "string"},
                            "medicoes": {
                                "bsonType": "array",
                                "items": {
                                    "bsonType": "object",
                                    "required": ["cod_pais", "ano", "valor"],
                                    "properties": {
                                        "cod_pais": {"bsonType": "string"},
                                        "ano": {"bsonType": "int"},
                                        "valor": {"bsonType": "double"}
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }

    try:
        db.create_collection("energia", validator=schema, validationAction=v_action, validationLevel=v_level)
    except Exception as e:
        print(f"Erro ao criar a coleção: {e}")


In [ ]:
def create_tables(db, col, v_action="warn", v_level="moderate"):
    if v_action not in ["warn", "error"]:
        raise ValueError("Ação de validação inválida. Use 'warn' ou 'error'.")
    if v_level not in ["strict", "moderate"]:
        raise ValueError("Nível de validação inválido. Use 'off', 'strict' ou 'moderate'.")
    if col == 'pais':
        create_pais_collection(db, v_action, v_level)
    elif col == 'emissaoGas':
        create_emissao_gas_collection(db, v_action, v_level)
    elif col == 'energia':
        create_energia_collection(db, v_action, v_level)
    else:
        raise ValueError(f"Nome de coleção inválido: {col}")

In [15]:
def create_country_document(cursor):
	query = 'SELECT * FROM PAIS;'
	cursor.execute(query)
	rows = cursor.fetchall()
	paises = []
	for row in rows:
		paises.append({
						'sigla': row[0],
						'nome': row[1],
						'continente': row[2],
						'estatisticas': []
					})
	# Grupos
	for pais in paises:
		# if pais['sigla'] != 'AFG':
		# 	continue
		query = f'''SELECT SIGLA
		FROM
		GRUPO G INNER JOIN PAIS_GRUPO PG
			ON G.ID_GRUPO = PG.GRUPO_ID_GRUPO
		WHERE PAIS_CODIGO = '{pais['sigla']}';'''
		cursor.execute(query)
		rows = cursor.fetchall()
		grupos = [row[0] for row in rows]
		if len(grupos) > 0:
			pais['grupos'] = grupos

		query = f'''SELECT ANO,
		VALOR_USD
		FROM PIB
		WHERE CODIGO_PAIS = '{pais['sigla']}'
		ORDER BY ANO;'''
		cursor.execute(query)
		rows = cursor.fetchall()

		for row in rows:
			ano, pib = row
			# print(ano, pib, pais['sigla'])
			geografias = []
			# Áreas e populações urbana, rural e total
			subquery = f'''SELECT TIPO,
			NUM_HABITANTES,
			AREA_KM2
			FROM
			AREA A INNER JOIN DEMOGRAFIA D
				ON D.AREA_ID = A.ID
			INNER JOIN DISTRIBUICAO_TERRITORIAL DT
				ON DT.AREA_ID = A.ID AND DT.PAIS_CODIGO = D.PAIS_CODIGO AND DT.ANO = D.ANO
			WHERE DT.PAIS_CODIGO = '{pais['sigla']}'
				AND DT.ANO = {ano}
			ORDER BY DT.ANO, TIPO;'''
			cursor.execute(subquery)
			subrows = cursor.fetchall()
			for sr in subrows:
				geografia = {
					'area_tipo': sr[0],
					'num_habitantes': sr[1],
					'area_km2': sr[2]
				}
				geografias.append(geografia)
			
			# Área florestal
			subquery = f'''SELECT TIPO,
			AREA_KM2
			FROM
			AREA A INNER JOIN DISTRIBUICAO_TERRITORIAL DT
				ON DT.AREA_ID = A.ID
			WHERE DT.PAIS_CODIGO = '{pais['sigla']}'
				AND DT.ANO = {ano}
			AND TIPO = 'Forest'
			ORDER BY DT.ANO, TIPO;'''
			cursor.execute(subquery)
			subrow = cursor.fetchall()
			for sr in subrow:
				geografia = {
					'area_tipo': sr[0],
					'area_km2': sr[1]
				}
				geografias.append(geografia)

			est = {
				'ano': row[0],
				'pib': row[1]
			}
			if geografias:
				est['geografias'] = geografias
			pais['estatisticas'].append(est)
	return paises


In [16]:
def create_gas_document(cursor):
	emissoes = []
	query = 'SELECT * FROM GAS;'
	cursor.execute(query)
	rows = cursor.fetchall()

	for row in rows:
		id, nome, formula = row
		emissao = {
			'nome': nome,
			'formula': formula,
			'medicoes': []
		}

		subquery = f'''SELECT PAIS_CODIGO,
		ANO,
		EMISSAO_MTCO2
		FROM EMISSAO_GAS
		WHERE GAS_ID_GAS = {id};'''
		cursor.execute(subquery)
		subrows = cursor.fetchall()

		for sr in subrows:
			cod_pais, ano, valor = sr
			emissao['medicoes'].append({
				'cod_pais': cod_pais,
				'ano': ano,
				'valor_mtco2': valor
			})
		
		emissoes.append(emissao)

	return emissoes

In [17]:
def create_energy_document(cursor):
	energias = []

	query = 'SELECT * FROM INTERACAO_ENERGETICA;'
	cursor.execute(query)
	rows = cursor.fetchall()

	for row in rows:
		id_interacao, interacao, unidade = row
		energia = {
			'interacao': interacao,
			'unidade': unidade,
			'fontes': []
		}
		
		query = 'SELECT * FROM ENERGIA;'
		cursor.execute(query)
		subrows = cursor.fetchall()

		for sr in subrows:
			id_energia, nome = sr
			fonte = {
				'nome': nome,
				'medicoes': []
			}
			query = f'''SELECT CODIGO_PAIS,
			ANO,
			VALOR
			FROM RELATORIO_ENERGETICO
			WHERE ID_INTERACAO = {id_interacao}
				AND ID_ENERGIA = {id_energia};'''
			cursor.execute(query)
			registros = cursor.fetchall()

			for reg in registros:
				cod_pais, ano, valor = reg
				fonte['medicoes'].append({
					'cod_pais': cod_pais,
					'ano': ano,
					'valor': valor
				})

			energia['fontes'].append(fonte)

		energias.append(energia)

	return energias

In [ ]:
def populate_mongo(db):
	pg_conn = create_pg_connection()
	cursor = pg_conn.cursor()

	entries = {'pais': create_country_document(cursor),
		'emissaoGas': create_gas_document(cursor),
		'energia': create_energy_document(cursor)}
	
	end_pg_connection(pg_conn)
	
	for col in entries.keys():

		if col not in db.list_collection_names():
			db.create_collection(col)

		create_tables(db, col)
		db[col].delete_many({})
		db[col].insert_many(entries[col])

		print(f'> Coleção {col} <')
		# Exibindo os documentos inseridos
		for doc in db[col].find({}, {'_id': 0}):
			pprint(doc)

In [ ]:
populate_tables = False
client = create_mongo_connection()
db = client["projeto_mc536"]

if populate_tables:
	populate_mongo(db)

end_mongo_connection(client)

---- Conexão bem sucedida ----
---- Conexão encerrada -----


In [ ]:
client = create_mongo_connection()
db = client["projeto_mc536"]

query = db.energia.aggregate([
    {"$match": {"tipo_interacao": "generation"}},
    {"$unwind": "$fontes"},
    {"$unwind": "$fontes.medicoes"},
    {"$match": {"fontes.medicoes.ano": 2000}},
    {"$group": {
        "_id": "$fontes.medicoes.cod_pais",
        "energia_total": {"$sum": "$fontes.medicoes.valor"},
        "unidade": {"$first": "$unidade"}
    }},
    {"$project": {"_id": 0, "cod_pais": "$_id", "energia_total": 1, "unidade": 1}}
])

pprint(list(query))

query = db['energia'].find({}, {'_id':0, 'tipo_interacao':1})

for doc in query:
	pprint(doc)

---- Conexão bem sucedida ----
[]
{}
{}
{}
{}
{}
